In [542]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from Utils import ErrorMetricsUtils as err
from Utils import CorrectnessMetricUtils as cmu
#from Utils import KernelsUtils as ker
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from csv import reader
import random


class LinearModel():
    
    def __init__ (self, loss = "rmse", model_type = "linear", learning_type = "constant", alpha = 0.001, epsilon = 0.1,):
        self.alpha = alpha
        self.loss = loss
        self.epsilon = epsilon
        self.learning_type = learning_type
        self.theta = np.zeros(1)
        self.poly = PolynomialFeatures(1)
        self.scaler = StandardScaler()
        self.fitted = False
        self.model_type = model_type
    
    # General linear regression
    def fit (self, train_set):
        X_train = np.asarray(train_set)[:, :-1]
        y_train = np.asarray(train_set)[:, -1]
        #X_test = np.array(test_set)[:, :-1]
        #Xk_train = np.ones((X_train.shape[0], X_train.shape[1] + 1))
        #Xk_test = np.ones((X_test.shape[0], X_test.shape[1] + 1))
        XS_train = self.scaler.fit_transform(X_train)
        #XS_test = scaler.fit_transform(X_test)
        Xk_train = self.poly.fit_transform(XS_train)
        #Xk_test = poly.fit_transform(XS_test)
        # choose convergence method
        self.theta = bch_grad_descent (Xk_train, y_train, self.model_type, self.loss, self.epsilon, self.alpha, self.learning_type) 
        #print (Xk_test.shape, " ", Xk_train.shape, " ", theta.shape)
        self.fitted = True
        return self.theta
    
    def predict (self, test_set):
        X_test = np.asarray(test_set)[:, :-1]
        XS_test = self.scaler.fit_transform(X_test)
        Xk_test = self.poly.fit_transform(XS_test)
        if (self.check_fitted(Xk_test)): 
            return Xk_test @ self.theta
        else: 
            return -1 # some error statement
    
    def check_fitted (self, Xk_test):
        #print (self.fitted, self.theta.shape, Xk_test.shape[1])
        return self.fitted and self.theta.shape[0] == Xk_test.shape[1]
        

In [543]:
# Batch Gradient Descent
def bch_grad_descent (Xk_train, y_train, lin_type, loss_fn, epsilon, alpha, learning_type):
    theta = np.zeros(Xk_train.shape[1]) # Xk_train is assumed to be a numpy array
    if (lin_type == "perceptron"):
        g = lambda x: np.piecewise(x, [x < 0, x >= 0], [0, 1])
        return _bch_grad_descent (Xk_train, y_train, theta, loss_fn, epsilon, alpha, learning_type, g)
    elif (lin_type == "logistic"):
        return _bch_grad_descent (Xk_train, y_train, theta, loss_fn, epsilon, alpha, learning_type, lambda x: 1 / (1 + np.exp(-x)))
    else:
        return _bch_grad_descent (Xk_train, y_train, theta, loss_fn, epsilon, alpha, learning_type, lambda x: x)

# Standard ot Batch Gradient Descent Helper (Divide by loss function)
def _bch_grad_descent (Xk_train, y_train, theta, loss_fn, epsilon, alpha, learning_type, g):
    if (loss_fn == "rmse"):
        return bch_grad_descent_rmse (Xk_train, y_train, theta, epsilon, alpha, learning_type, g)
    if (loss_fn == "kld"):
        return bch_grad_descent_kld (Xk_train, y_train, theta, epsilon, alpha, learning_type, g)
    else:
        return bch_grad_descent_mse (Xk_train, y_train, theta, epsilon, alpha, learning_type, g)

# Standard or Batch Gradient Descent under the Mean Squared Error loss function 
def bch_grad_descent_mse (Xk_train, y_train, theta, epsilon, alpha, lt, g):
    hypothesis = g(Xk_train@theta) # matrix multiplication of feature array with parameter array
    #print ("Initial Training Labels:", y_train, " with size ", y_train.shape)
    #print ("The feature set is: ", Xk_train, " with size ", Xk_train.shape)
    i = 0
    while (True): 
        mse_err = err.mse_calc(y_train, hypothesis)
        neg_gradient = (Xk_train.T)@(y_train-hypothesis)
        if (i % 1000 == 0):
            #print ("Interation: ", i)
            #print ("The hypothesis after ", i, " steps is ", Xk_train@theta)
            #print ("The parameters after ", i , " steps is ", theta)
            print (mse_err)
            #print (np.mean(y_train-hypothesis))
            #print ("Difference between prediction and labels after ", i, " steps is ", y_train-hypothesis)
            #print ("The gradient after ", i, " steps is ", neg_gradient)
        if (lt == "optimal"):
            beta = alpha/np.linalg.norm(neg_gradient)
        else: beta = alpha
        change = beta*neg_gradient
        if (np.linalg.norm(change) < epsilon): break
        theta = theta + change # using matrix operations
        i = i + 1
        hypothesis = g(Xk_train@theta)
    #print (mse_err)
    print ("converged after ", i, " iterations")
    return theta

# Standard or Batch Gradient Descent under the Mean Absolute Error loss function
def bch_grad_descent_rmse (Xk_train, y_train, theta, epsilon, alpha, lt, g):
    print(1)
    hypothesis = g(Xk_train @ theta) # matrix multiplication of feature array with parameter array
    while (True):   
        rmse_err = err.rmse_calc (y_train, hypothesis)
        neg_gradient = (Xk_train.T)@(y_train-hypothesis)
        print (rmse_err)
        if (lt == "optimal"):
            beta = alpha/np.linalg.norm(neg_gradient)
        else: beta = alpha
        change = (beta/rmse_err)*neg_gradient
        if (np.linalg.norm(change) < epsilon): break
        theta = theta + change # essentially MSE equation divided by RMSE
        hypothesis = g(Xk_train@theta)
    return theta

def bch_grad_descent_kld (Xk_train, y_train, theta, epsilon, alpha, lt, g):
    hypothesis = g(Xk_train @ theta) # matrix multiplication of feature array with parameter array
    # fix problems with zero denominator and negative values
    while (True):
        neg_gradient = sum ((y[i]/hypothesis[i])*X_theta[i] for i in range (len(y_train)))
        if (lt == "optimal"):
            beta = alpha/np.linalg.norm(neg_gradient)
        else: beta = alpha
        change = beta*gradient
        if (np.linalg.norm(change) < epsilon): break
        theta = theta + change
        hypothesis = g(Xk_train@theta)
    return theta

#def descent_step (theta0, neg_gradient, alpha, cur_err):
#    beta = alpha/np.mean(neg_gradient**2)
#    #if (curr_err < 5): beta = 1e-2*beta
#    #if (curr_err < 4): beta = 1e-2*beta
#    return theta + beta*neg_gradient

In [544]:
# Stochastic Gradient Descent
def stch_grad_descent (Xk_train, y_train, lin_type, loss_fn, epsilon, alpha, learning_type):
    theta = np.zeros(Xk_train.shape[1]) # Xk_train is assumed to be a numpy array
    batch_size = 49
    if (lin_type == "perceptron"):
        g = lambda x: np.piecewise(x, [x < 0, x >= 0], [0, 1])
        return _stch_grad_descent (Xk_train, y_train, theta, loss_fn, epsilon, alpha, learning_type, batch_size, g)
    elif (lin_type == "logistic"):
        g = lambda x: 1 / (1 + np.exp(-x))
        return _stch_grad_descent (Xk_train, y_train, theta, loss_fn, epsilon, alpha, learning_type, batch_size, g)
    else:
        return _stch_grad_descent (Xk_train, y_train, theta, loss_fn, epsilon, alpha, learning_type, batch_size, lambda x: x)


# Stochastic Gradient Descent Helper (Divide by log function)
def _stch_grad_descent (Xk_train, y_train, theta, loss_fn, epsilon, alpha, learning_type, batch_size, g):
    if (loss_fn == "rmse"):
        return stch_grad_descent_rmse (Xk_train, y_train, theta, epsilon, alpha, batch_size, learning_type, g)
    if (loss_fn == "kld"):
        return stch_grad_descent_kld (Xk_train, y_train, theta, epsilon, alpha, batch_size, learning_type, g)
    else:
        return stch_grad_descent_mse (Xk_train, y_train, theta, epsilon, alpha, batch_size, learning_type, g)

def get_batch (Xk_train, y_train, batch_size):
    random.seed(3) # replace with system time?
    index = random.randrange(Xk_train.shape[0] - batch_size)
    return Xk_train[index:index+batch_size, :], y_train[index:index+batch_size]

# Stochastic Gradient Descent under the Mean Squared Error loss function 
def stch_grad_descent_mse (Xk_train, y_train, theta, epsilon, alpha, batch_size, lt, g):
    Xk_train_sgd, y_train_sgd = get_batch (Xk_train, y_train, batch_size)
    hypothesis = g(Xk_train_sgd@theta) # matrix multiplication of feature array with parameter array
    i = 0
    while (True): 
        mse_err = err.mse_calc(y_train_sgd, hypothesis)
        neg_gradient = (Xk_train_sgd.T)@(y_train_sgd-hypothesis)
        if (i % 10000 == 0):
            #print ("Interation: ", i)
            print (mse_err)
        if (lt == "optimal"):
            beta = alpha/np.linalg.norm(neg_gradient)
        else: beta = alpha
        change = beta*neg_gradient
        if (np.linalg.norm(change) < epsilon): break
        theta = theta + change # using matrix operations
        i = i + 1
        Xk_train_sgd, y_train_sgd = get_batch (Xk_train, y_train, batch_size)
        hypothesis = g(Xk_train_sgd@theta)
    print (mse_err)
    print ("converged after ", i, " iterations")
    return theta

# Stochastic Gradient Descent under the Mean Absolute Error loss function
def stch_grad_descent_rmse (Xk_train, y_train, theta, epsilon, alpha, batch_size, lt, g):
    Xk_train_sgd, y_train_sgd = get_batch (Xk_train, y_train, batch_size) # take out batch_size samples
    hypothesis = g(Xk_train_sgd @ theta) # matrix multiplication of feature array with parameter array
    while (True):
        rmse_err = err.rmse_calc (y_train_sgd, hypothesis)
        print (rmse_err)
        neg_gradient = (Xk_train_sgd.T)@(y_train_sgd-hypothesis)
        if (lt == "optimal"):
            beta = alpha/np.linalg.norm(neg_gradient)
        else: beta = alpha
        change = (beta/rmse_err)*(neg_gradient)
        if (np.linalg.norm(change) < epsilon): break
        theta = theta + change # essentially MSE equation divided by RMSE
        Xk_train_sgd, y_train_sgd = get_batch (Xk_train, y_train, batch_size)
        hypothesis = g(Xk_train_sgd@theta)
    print (mse_err)
    print ("converged after ", i, " iterations")
    return theta

In [545]:
# Newton's Method
def newton_method (Xk_train, y_train, lin_type, loss_fn, epsilon, alpha, learning_type):
    theta = np.zeros(Xk_train.shape[1]) # Xk_train is assumed to be a numpy array
    if (lin_type == "perceptron"):
        g = lambda x: np.piecewise(x, [x < 0, x >= 0], [0, 1])
        return newton_method_mse (Xk_train, y_train, theta, epsilon, alpha, g)
    elif (lin_type == "logistic"):
        return newton_method_mse (Xk_train, y_train, theta, epsilon, alpha, lambda x: 1 / (1 + np.exp(-x)))
    else:
        return newton_method_mse (Xk_train, y_train, theta, epsilon, alpha, lambda x: x)

def newton_method_mse (Xk_train, y_train, theta, epsilon, alpha, g):
    i = 0;
    while (True):
        neg_gradient = Xk_train.T @ (y_train - g (Xk_train @ theta))
        hinv = np.linalg.pinv(*(Xk_train.T @ Xk_train))
        change = hinv @ neg_gradient
        #print ("{}th iteration".format(i + 1))
        if (np.linalg.norm (change, ord = 1) < epsilon): 
            #print("too small a change to proceed");
            break
        theta = theta + change
        i += 1
    return theta

# Normal Equations to solve regression (assumed MSE loss function)
def normal_eqn (Xk_train, y_train, loss_fn, epsilon, alpha, learning_type):
    return np.linalg.pinv(Xk_train)@y_train

In [546]:
# Get CSV file
def get_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        data = reader(file)
        for row in data:
            if not row:
                continue
            dataset.append(row)
    return dataset

#String to float columnwise
def str_to_float_col(dataset, col):
    for row in dataset:
        row[col] = float(row[col].strip())

def get_csv2 (filename):
    op = np.genfromtxt(filename, delimiter=',')
    op = op[:, 1:]
    return op.astype(np.float)
        
# Split dataset into n folds
def crossval_split(dataset, n_folds):
    split = list()
    dataset_copy = list(dataset)
    fold_dim = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_dim:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        split.append(fold)
    return split

# Algo evaluation by cross validation split
def eval_algo_cross_val (dataset, n_folds, *args):
    folds = crossval_split(dataset, n_folds)
    mseScores = list()
    maeScores = list()
    rmseScores = list()
    klScores = list()
    jsScores = list()
    ceScores = list()
    r2Scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = fold
        algo = LinearModel() # how to generalize this
        algo.fit(train_set)
        predicted = algo.predict(test_set)
        actual = [row[-1] for row in fold]
        mse = err.mse_calc(actual, predicted)
        mae = err.mae_calc(actual, predicted)
        rmse = err.rmse_calc(actual, predicted)
        #kl_div = err.kl_div_calc(actual, predicted)
        #js_div = err.js_div_calc(actual, predicted)
        #cross_entropy = err.cross_entropy_calc(actual, predicted)
        r2 = err.r2_calc(actual, predicted)
        mseScores.append(mse)
        maeScores.append(mae)
        rmseScores.append(rmse)
        #klScores.append(kl_div)
        #jsScores.append(js_div)
        #ceScores.append(cross_entropy)
        r2Scores.append(r2)
    return mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores

# Train over entire set and report training Error
def eval_algo_train (dataset, n_folds, *args):
    #new_d = list ()
    #new_d.append(dataset[0])
    #new_d.append(dataset[1])
    #new_d.append(dataset[2])
    #new_d.append(dataset[3])
    #new_d.append(dataset[4])
    mseScores = list()
    maeScores = list()
    rmseScores = list()
    klScores = list()
    jsScores = list()
    ceScores = list()
    r2Scores = list()
    train_set = list(dataset)
    #test_set = list()
    #for row in dataset:
    #    row_copy = list(row)
    #    test_set.append(row_copy)
    #    row_copy[-1] = None
    algo = LinearModel()
    algo.fit(train_set)
    predicted = algo.predict(train_set)
    actual = [row[-1] for row in dataset]
    mse = err.mse_calc(actual, predicted)
    mae = err.mae_calc(actual, predicted)
    rmse = err.rmse_calc(actual, predicted)
    #kl_div = err.kl_div_calc(actual, predicted)
    #js_div = err.js_div_calc(actual, predicted)
    #cross_entropy = err.cross_entropy_calc(actual, predicted)
    r2 = err.r2_calc(actual, predicted)
    mseScores.append(mse)
    maeScores.append(mae)
    rmseScores.append(rmse)
    #klScores.append(kl_div)
    #jsScores.append(js_div)
    #ceScores.append(cross_entropy)
    r2Scores.append(r2)
    #print (len(predicted), " ", len(actual))
    return mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores

# evaluate algorithm
random.seed(1)
filename = 'data/regression.csv'
dataset = get_csv(filename)
dataset.remove(dataset[0]) # remove headings
for i in range(len(dataset[0])): # convert dataset to float columnwise
    str_to_float_col(dataset, i) 
#print (dataset[2])
    
n_folds = 5
print('Running k-fold cross validation with 5 folds')
mseScores, maeScores, rmseScores, klScores, jsScores, ceScores, r2Scores = eval_algo_cross_val (dataset, n_folds)
print('MSE Loss: %s' % mseScores)
print('MAE Loss: %s' % mseScores)
print('RMSE Loss: %s' % rmseScores)
print('R squared Loss: %s' % r2Scores)
print('KL Divergence: %s' % klScores)
print('JS Divergence: %s' % jsScores)
print('Cross Entropy: %s' % ceScores)
print('Mean MSE Loss: %s' % (sum(mseScores)/float(len(mseScores))))
print('Mean MAE Loss: %s' % (sum(mseScores)/float(len(mseScores))))
print('Mean RMSE Loss: %s' % (sum(rmseScores)/float(len(rmseScores))))
print('Mean R squared Loss: %s' % (sum(r2Scores)/float(len(r2Scores))))
#print('Mean KL Divergence: %s' % (sum(klScores)/float(len(klScores))))
#print('Mean JS Divergence: %s' % (sum(jsScores)/float(len(jsScores))))
#print('Mean Cross Entropy: %s' % (sum(ceScores)/float(len(ceScores))))

Running k-fold cross validation with 5 folds
1
17.216235070421174
16.340936416439778
15.47283214284329
14.612281107867561
13.759691492724848
12.91553335592913
12.080355448122807
11.254808026815878
10.439674235420231
9.635913877392925
8.844725378868521
8.067634789388212
7.306625409652608
6.564328817047575
5.8443082417357894
5.151477106509012
4.492699897812412
3.8775838384511556
3.3192796540185627
2.834525033335776
2.4410111806424286
2.14978410844609
1.9554066301132422
1.8353699848348801
1.7628187620000488
1.71795461143656
1.6893441966160154
1.6707588185400202
1.658612714615736
1
17.508426542667962
16.6332974645103
15.765210924923673
14.904543148513875
14.051722730765288
13.207243492754971
12.371681528310038
11.545718089607643
10.730170703996254
9.926036039027963
9.134549734495465
8.357270995011202
7.596203584411236
6.853970405922881
6.134066108179415
5.441219214620085
4.781893535611963
4.164919754403832
3.6020953026938236
3.1081705404977074
2.6988663568741846
2.3852665753673774
2.165882